In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['font.sans-serif'] = 'Helvetica'
import seaborn as sns
sns.set_context("paper")
import os 
import glob
import statsmodels.api as sm
from pathlib import Path 
from scipy import stats
import pingouin as pg
import scipy

#some helper fxn's for formatting
from math import floor, log10

def sig_figs(x: float, precision: int):
    """
    Rounds a number to number of significant figures
    Parameters:
    - x - the number to be rounded
    - precision (integer) - the number of significant figures
    Returns:
    - float
    """

    x = float(x)
    precision = int(precision)

    return round(x, -int(floor(log10(abs(x)))) + (precision - 1))

# DID Dataloader

In [ ]:
file_to_open = os.path.join(os.getcwd(), 'DrinkingData.xlsx')
print(file_to_open)

DID = pd.read_excel(file_to_open)

#calc intakes
DID.loc[DID['Fluid'] == "Alcohol", "Intake"] = (((DID['InitialWeight']-DID['FinalWeight'])/0.97336)*0.789*0.2)/(DID['Weight']*0.001)
DID.loc[DID['Fluid'] == "Water", "Intake"] = (((DID['InitialWeight']-DID['FinalWeight'])/1)*1)/(DID['Weight']*0.001)

#calc water intake from previous 24 hours
DID['Water_Intake'] = (((DID['WaterBottleWeightStart']-DID['WaterBottleWeightStop'])/1)*1)/(DID['Weight']*0.001)

#Add DID Week
DID.loc[DID['Session'] <= 0, "DIDWeek"] = 0
DID.loc[(DID['Session'] >= 1) & (DID['Session'] <= 5), "DIDWeek"] = 1
DID.loc[(DID['Session'] >= 6) & (DID['Session'] <= 10), "DIDWeek"] = 2
DID.loc[(DID['Session'] >= 11) & (DID['Session'] <= 15), "DIDWeek"] = 3
DID.loc[(DID['Session'] >= 16) & (DID['Session'] <= 20), "DIDWeek"] = 4
DID.loc[(DID['Session'] >= 21) & (DID['Session'] <= 25), "DIDWeek"] = 5
DID.loc[(DID['Session'] >= 26) & (DID['Session'] <= 30), "DIDWeek"] = 6

#Add Cohort Data
DID.loc[DID['ID'] == "P1", "Cohort"] = 1
DID.loc[DID['ID'] == "P2", "Cohort"] = 1
DID.loc[DID['ID'] == "P3", "Cohort"] = 1
DID.loc[DID['ID'] == "P4", "Cohort"] = 1

DID.loc[DID['ID'] == "P5", "Cohort"] = 2
DID.loc[DID['ID'] == "P6", "Cohort"] = 2
DID.loc[DID['ID'] == "P7", "Cohort"] = 2
DID.loc[DID['ID'] == "P8", "Cohort"] = 2
DID.loc[DID['ID'] == "P9", "Cohort"] = 2

DID.loc[DID['ID'] == "P10", "Cohort"] = 3
DID.loc[DID['ID'] == "P11", "Cohort"] = 3
DID.loc[DID['ID'] == "P12", "Cohort"] = 3
DID.loc[DID['ID'] == "P13", "Cohort"] = 3
DID.loc[DID['ID'] == "P14", "Cohort"] = 3

for i in DID['ID'].unique():
    DID.loc[DID['ID'] == i, 'Water_Intake'] = DID.loc[DID['ID'] == i, 'Water_Intake'].shift()
    DID.loc[DID['ID'] == i, 'Date_Parsed'] = pd.to_datetime(DID.loc[DID['ID'] == i, 'Date'])
    DID.loc[DID['ID'] == i, 'Date_Diff'] = DID.loc[DID['ID'] == i, 'Date_Parsed'].diff()

DID.loc[(DID['Session'] == 15) & (DID['Cohort'] == 1), "Water_Intake"] = np.nan ##Water Bottle tops chewed off so evaporation error

DID['Previous_Water'] = DID['Water_Intake']/(DID['Date_Diff'].dt.days)

DID.loc[DID['Previous_Water'] > 275, 'Previous_Water'] = np.nan #filter out leaks

#Clean up raw data columns
DID = DID.drop(['InitialWeight'], axis = 1)
DID = DID.drop(['FinalWeight'], axis = 1)

#remove datapoints in notes
DID.loc[(DID['ID'] == "P14") & (DID['Session'] == 19), 'Intake'] = np.nan
DID.loc[(DID['ID'] == "P14") & (DID['Session'] == 20), 'Intake'] = np.nan

DID.head()

# Weights

In [ ]:
f, ax = plt.subplots(figsize=(12,8))
sns.despine()

sns.lineplot(data = DID, x = 'Session', y = "Weight", legend=False, errorbar='se')
sns.lineplot(data = DID, x = 'Session', y = "Weight", hue = "ID", legend=False, alpha = 0.5)

plt.ylabel('Mouse Weight (g)')
plt.xlabel('Drinking Session')

plt.ylim(18,32)

plt.savefig('Weights.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

In [ ]:
f, ax = plt.subplots(figsize=(12,8))
sns.despine()

data = DID.loc[DID['Session'] > 0]

sns.lineplot(data = data, x = 'Session', y = "Weight", hue = "Sex", errorbar='se', hue_order=['Male','Female'], palette=['#023047', '#FB8500'])


handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:], frameon=False, loc = 'upper left')

plt.ylabel('Mouse Weight (g)')
plt.xlabel('Drinking Session')

plt.ylim(18,32)

plt.savefig('Weight_Sex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv="Weight", within='Session', between='Sex', subject='ID', data=data).round(4)

# Intakes

### Water

In [ ]:
f, ax = plt.subplots(figsize=(24,8))
sns.despine()
var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Water")]

sns.pointplot(data = data, x = 'Session', y = var, hue = "ID", errorbar = 'se')

plt.ylabel('Water Intake (g/kg)')
plt.xlabel('Drinking Session')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 2)

plt.show()
plt.clf()

pg.rm_anova(dv="Intake", within='Session', subject='ID', data=data)

In [ ]:
f, ax = plt.subplots(figsize=(12,8))
sns.despine()
var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Water")]

sns.stripplot(data = data, x = 'Session', y = var, hue = "ID", jitter = True, edgecolor='gray', alpha =0.5)
sns.pointplot(data = data, x = 'Session', y = var, hue = "Fluid", errorbar = 'se')

plt.ylabel('Water Intake (g/kg)')
plt.xlabel('Drinking Session')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 2)

plt.ylim(0,100)

plt.savefig('Water_Intakes_ALL.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

#pg.rm_anova(dv="Intake", within='Session', subject='ID', data=data, effsize="ng2").round(4)

In [ ]:
f, ax = plt.subplots(figsize=(12,8))
sns.despine()
var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Water")]

sns.stripplot(data = data, x = 'Session', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5)
sns.pointplot(data = data, x = 'Session', y = var, hue = "Sex", errorbar = 'se')

plt.ylabel('Water Intake (g/kg)')
plt.xlabel('Drinking Session')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 2)

plt.ylim(0,100)

plt.savefig('Water_Intakes_ALL_SEX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv="Intake", within='Session', between='Sex', subject='ID', data=data, effsize="ng2").round(4)

In [ ]:
f, ax = plt.subplots(figsize=(12,8))
sns.despine()
var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Water")]

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "ID", jitter = True, edgecolor='gray', alpha =0.5, linewidth=0)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Fluid", errorbar= 'se', estimator= np.mean)

plt.ylabel('Water Intake (g/kg)')
plt.xlabel('Drinking Week')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper right')

plt.ylim([0,100])

plt.savefig('WaterxWeek_ALL.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv="Intake", within='DIDWeek', subject='ID', data=data)

In [ ]:
posthoc = pg.pairwise_tests(dv="Intake", within='DIDWeek', subject='ID', padjust='sidak', data=data)
posthoc.loc[posthoc['p-corr'] < 0.05]

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth/1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Water") & (DID['Session'] > 0)]

sns.stripplot(data = data, x = 'Session', y = var, hue = "ID", jitter = True, edgecolor='gray', alpha =0.5, palette='dark:#1B263B', size = 1)
sns.pointplot(data = data, x = 'Session', y = var, hue = "Fluid", errorbar = 'se', palette='dark:#1B263B', scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Intake (g/kg)', fontsize = 5)
plt.xlabel('Drinking Session', fontsize = 5)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 2)

plt.ylim(0,100)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('WaterIntake.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv="Intake", within='Session', subject='ID', data=data)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth/1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Water") & (DID['Session'] > 0)]

sns.stripplot(data = data, x = 'Session', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1)
sns.pointplot(data = data, x = 'Session', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Intake (g/kg)', size = 5)
plt.xlabel('Drinking Session', size = 5)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.ylim(0,100)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)
plt.tight_layout()
plt.savefig('WaterIntake_Sex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, within='Session', between='Sex', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth/1.6
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Water") & (DID['Session'] > 0)]

data['DIDWeek'] = data['DIDWeek'].astype('int')

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "ID", jitter = True, edgecolor='gray', alpha =0.5, palette='dark:#1B263B', size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Fluid", errorbar= 'se', estimator= np.mean, palette='dark:#1B263B', scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Intake (g/kg)', size = 5)
plt.xlabel('Drinking Week', size = 5)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.ylim([0,100])

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('WaterIntakexWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv="Intake", within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth/1.6
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Water") & (DID['Session'] > 0)]

data['DIDWeek'] = data['DIDWeek'].astype('int')

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Intake (g/kg)', size = 5)
plt.xlabel('Drinking Week', size = 5)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.ylim([0,100])

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('WaterIntakexWeekxSex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv="Intake", within='DIDWeek', between='Sex', subject='ID', data=data).round(4)

### Alcohol

In [ ]:
f, ax = plt.subplots(figsize=(24,8))
sns.despine()
var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Alcohol")]

sns.pointplot(data = data, x = 'Session', y = var, hue = "ID", errorbar = 'se')

plt.ylabel('Alcohol Intake (g/kg)')
plt.xlabel('Drinking Session')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 2)

plt.show()
plt.clf()

pg.rm_anova(dv="Intake", within='Session', subject='ID', data=data)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth/1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = 'Intake'

data= DID.loc[(DID['Fluid'] == "Alcohol")]

sns.stripplot(data = data, x = 'Session', y = var, hue = "ID", jitter = True, edgecolor='gray', alpha =0.5, palette='dark:#00a170', size = 1)
sns.pointplot(data = data, x = 'Session', y = var, errorbar = 'se', color = "#00a170", scale = 0.5, errwidth = 0.5)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[0:0], labels[:], frameon=False, loc = 2)

plt.ylabel('Alcohol Intake (g/kg)\n', size = 5)
plt.xlabel('Drinking Session', size = 5)

plt.ylim(0,8)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('AlcoholIntakes.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv="Intake", within='Session', subject='ID', data=data, effsize="ng2")

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth/1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = 'Intake'

data= DID.loc[DID['Fluid'] == "Alcohol"]

sns.stripplot(data = data, x = 'Session', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1)
sns.pointplot(data = data, x = 'Session', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Alcohol Intake (g/kg)\n', size = 5)
plt.xlabel('Drinking Session', size = 5)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.ylim(0,8)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('AlcoholxSex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv="Intake", within='Session', between='Sex', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth/1.6
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = 'Intake'

data= DID.loc[DID['Fluid'] == "Alcohol"]

data['DIDWeek'] = data['DIDWeek'].astype('int')

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "ID", jitter = True, edgecolor='gray', alpha =0.5, palette='dark:#00a170', size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, errorbar= 'se', estimator= np.mean, color = '#00a170', scale = 0.5, errwidth = 0.5)

plt.ylabel('Alcohol Intake (g/kg)\n', size = 5)
plt.xlabel('Drinking Week', size = 5)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.ylim([0,8])

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('AlcoholxWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv="Intake", within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
posthoc = pg.pairwise_tests(dv="Intake", within='DIDWeek', subject='ID', padjust='sidak', data=data)
posthoc.loc[posthoc['p-corr'] < 0.05].round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth/1.6
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = 'Intake'

data= DID.loc[DID['Fluid'] == "Alcohol"]

data['DIDWeek'] = data['DIDWeek'].astype('int')

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Alcohol Intake (g/kg)\n', size = 5)
plt.xlabel('Drinking Session', size = 5)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.ylim([0,8])

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('AlcoholXweekXSex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv="Intake", within='DIDWeek', between='Sex', subject='ID', data=data).round(4)

In [ ]:
posthoc = pg.pairwise_tests(dv="Intake", within='DIDWeek', between='Sex', subject='ID', padjust='sidak', data=data)
posthoc.loc[posthoc['p-corr'] < 0.05]

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/5
figheight = figwidth
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = 'Previous_Water'

data = DID.loc[DID['Session'] > 0]
data['DIDWeek'] = data['DIDWeek'].astype('int')

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', scale = 0.5, errwidth = 0.5)

plt.ylabel('Inter-Session\nWater Intake (g/kg)', size = 5)
plt.xlabel('Drinking Week', size = 5)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.ylim([0,400])

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('IntasessionWater.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, within='DIDWeek', between='Sex', subject='ID', data=data).round(4)

In [ ]:
posthoc = pg.pairwise_tests(dv=var, within='DIDWeek', between='Sex', subject='ID', padjust='sidak', data=data)
posthoc.loc[posthoc['p-corr'] < 0.05]

# Microstructure Dataloader & Processing

In [ ]:
#get current working directory and then move into the CSVs folder
#wd = os.path.join(os.getcwd(), 'CSVs')

#for i in DID['ID'].unique():
#    print(i)
#    #define new basefolder to search based on current animal ID
#    base_folder = os.path.join(wd, str(i))
#    #pull all the CSVs for the current animal
#    for csv_file in glob.glob(str(base_folder) + "/*.CSV"):
#        print(csv_file)
#        csv = pd.read_csv(csv_file)
#        xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
#        csv.to_excel(xlsx_file, index=None, header=True)

In [ ]:
masterdrinker = pd.DataFrame()

#get current working directory and then move into the CSVs folder
wd = os.path.join(os.getcwd(), 'CSVs')

for i in DID['ID'].unique():
    print("Loading Data...", i)
    #define new basefolder to search based on current animal ID
    base_folder = os.path.join(wd, str(i))
    #pull all the XLSXs for the current animal
    filenames =  glob.glob(str(base_folder) + "/*.xlsx")
    #loop thru all the files for this animal and add them to a list that resets with each animal
    DIDs = []
    current_animal = []
    for filename in filenames:
        DIDs.append(pd.read_excel(filename, ))
    current_animal = pd.concat(DIDs, ignore_index=True)
    current_animal.columns = current_animal.columns.str.replace(' ', '') #remove spaces from column names
    current_animal = current_animal.drop(columns=['Shocks','LeftShockProbability', 'RightShockProbability']) #drop stupid columns
    current_animal['ID'] = i #assign a new column based on animal index
    current_animal['Sex'] = DID.loc[DID['ID'] == i, 'Sex'].iloc[0] #look up sex
    current_animal['Cohort'] = DID.loc[DID['ID'] == i, 'Cohort'].iloc[0] #look up cohort
    current_animal['Date'] = pd.to_datetime(current_animal['MM:DD:YYYYhh:mm:ss'], format='%m/%d/%Y %H:%M:%S') #convert to pandas datetime object

    current_animal = current_animal.drop(['MM:DD:YYYYhh:mm:ss','Device'], axis = 1) #drop old datetime and change in time columns

    for j in DID.loc[DID['ID'] == i, "Session"].unique():
        # does the lickometer data for the session that you're looking up even exist?
        if len(current_animal.loc[current_animal['Date'].dt.strftime('%m-%d-%Y') == DID.loc[(DID['ID'] == i) & (DID['Session'] == j), "Date"].dt.strftime('%m-%d-%Y').iloc[0]].copy()) != 0: 
            current_session = current_animal.loc[current_animal['Date'].dt.strftime('%m-%d-%Y') == DID.loc[(DID['ID'] == i) & (DID['Session'] == j), "Date"].dt.strftime('%m-%d-%Y').iloc[0]].copy()
            current_session['Session'] = j
            current_session['DIDWeek'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), "DIDWeek"].iloc[0]
            current_session['SessionLength'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), "SessionLength"].iloc[0]
            current_session['Fluid'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), "Fluid"].iloc[0]
            current_session['BottleSide'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), "BottleSide"].iloc[0]

            #if using an old version of lickometer software - this will convert old column tags to new versions
            if 'ElapsedTime' in current_animal.columns:
                current_session['ElapsedSecs'] = pd.to_timedelta(current_session['TimeElapsed'], unit='seconds') #convert change in time to correct type

            current_session['ElapsedSecs'] = pd.to_timedelta(current_session['ElapsedSecs'], unit='seconds') #convert change in time to correct type
            
            if((current_session['BottleSide'].iloc[0]) == "Left"):
                current_session['FluidCount'] = current_session['LeftCount']
                current_session['FluidDuration'] = current_session['LeftDuration']

            if((current_session['BottleSide'].iloc[0]) == "Right"):
                current_session['FluidCount'] = current_session['RightCount']
                current_session['FluidDuration'] = current_session['RightDuration']

            current_session['FluidCountDiff'] = current_session['FluidCount'].diff().fillna(0)
            current_session['FluidDurationDiff'] = current_session['FluidDuration'].diff().fillna(0)            

            current_session['VideoRecording'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), "VideoRecording"].iloc[0]
            current_session['FPRecording'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), "FPRecording"].iloc[0]
            current_session['Seconds'] = current_session['ElapsedSecs'].dt.seconds
            current_session['Minutes'] = current_session['ElapsedSecs'].dt.seconds/60 #convert TimeElapsed into minutes
            current_session['Hours'] = current_session['ElapsedSecs'].dt.seconds/3600 #convert TimeElapsed into hours

            current_session_cleaned = []
           
            current_session_cleaned = current_session.loc[current_session['ElapsedSecs'] <= pd.Timedelta(hours= current_session['SessionLength'].unique()[0])].copy()

            current_session_cleaned = current_session_cleaned.loc[current_session_cleaned['FluidCountDiff'] > 0]

            masterdrinker = pd.concat([masterdrinker, current_session_cleaned])            
        # there's no lickometer data for this DID session, so continue the loop and keep going
        else:
            continue

print( '\n Cleaning Data... \n')

for i in masterdrinker['ID'].unique():
    for j in masterdrinker.loc[(masterdrinker['ID'] == i), 'Session'].unique():
        model = sm.OLS(masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j), 'FluidDurationDiff'],
                       masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j), 'FluidCountDiff'])
        masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j), 'resid'] = model.fit().resid

masterdrinker = masterdrinker.loc[masterdrinker['resid'] < 3]
masterdrinker = masterdrinker.loc[masterdrinker['resid'] > -3]

masterdrinker = masterdrinker.loc[masterdrinker['FluidDurationDiff'] < 15]

masterdrinker = masterdrinker.dropna(axis=1)
masterdrinker.to_excel('masterdrinker.xlsx')
masterdrinker.head()

sns.relplot(data = masterdrinker, x = "FluidCountDiff", y = "FluidDurationDiff", hue = 'ID')

In [ ]:
%%time
for i in masterdrinker['ID'].unique():
    print(i)
    sessions = masterdrinker.loc[(masterdrinker['ID'] == i), 'Session'].unique()
    for j in sessions:
        #Total Licks
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'TotalEvents'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0), 'FluidCountDiff'].sum()
        #Total Lick Duration
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'TotalEventDuration'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0), 'FluidDurationDiff'].sum()
        #Number of Drinks
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Drinks'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0), 'FluidCountDiff'].count()
        #Mean Licks per Drink
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'MeanEventsperDrink'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0), 'FluidCountDiff'].mean()
        #Mean Duration per Drink
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'MeanEventDurattionperDrink'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0), 'FluidDurationDiff'].mean()
        #Latency to Drink
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Latency'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0), "ElapsedSecs"].iloc[0].seconds
        #Interdrink Interval
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'IDI'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0), "ElapsedSecs"].diff().dropna().mean().seconds

        #####################################################################
        #First 30 Minutes
        #####################################################################
        #Total Licks
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'TotalEvents30'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0) & (masterdrinker['ElapsedSecs'] <= pd.Timedelta(hours=0.5)), 'FluidCountDiff'].sum()
        #Total Lick Duration
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'TotalEventDuration30'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0) & (masterdrinker['ElapsedSecs'] <= pd.Timedelta(hours=0.5)), 'FluidDurationDiff'].sum()
        #Number of Drinks
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Drinks30'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0) & (masterdrinker['ElapsedSecs'] <= pd.Timedelta(hours=0.5)), 'FluidCountDiff'].count()
        #Mean Licks per Drink
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'MeanEventsperDrink30'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0) & (masterdrinker['ElapsedSecs'] <= pd.Timedelta(hours=0.5)), 'FluidCountDiff'].mean()
        #Mean Duration per Drink
        DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'MeanEventDurattionperDrink30'] = masterdrinker.loc[(masterdrinker['ID'] == i) & (masterdrinker['Session'] == j) & (masterdrinker['FluidCountDiff'] > 0) & (masterdrinker['ElapsedSecs'] <= pd.Timedelta(hours=0.5)), 'FluidDurationDiff'].mean()

DID.to_excel('DID.xlsx') 
DID.to_parquet('DID.parquet')     


# Microstructure Analysis

## Event Corrrelations

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth

data = DID.loc[DID['Fluid'] == "Water"]

var = 'TotalEvents'

sns.lmplot(x= var, y=  "Intake", data = data, hue = 'Sex', 
           ci = 68,
           height = figheight,
           hue_order=['Male','Female'],
           scatter_kws={'linewidths': 0, "s": 2.5},
           line_kws={"linewidth":0.5},
           legend = False
           )

plt.ylabel('Water Intake (g/kg)', size = 5)
plt.xlabel('Drinking Events', size = 5)

plt.ylim([0,120])

plt.tick_params(labelsize=5, width=0.5, length = 2)

plt.legend(frameon = False, loc = 'lower right', fontsize = 5, markerscale = 2)

stat_out = pg.corr(data['Intake'], data[var], method='shepherd')
pval = sig_figs(stat_out['p-val'].iloc[0],4)
rval = sig_figs(stat_out['r'].iloc[0],4)

plt.text(x= 1, y = 110,  s = (f"r = {rval}\np = {pval}"), size = 5)

plt.tight_layout()
plt.savefig('Water_Events.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

print(pval)
print(rval)

pg.ancova(data=data, dv='Intake', between='Sex', covar= var, effsize='np2')

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth

data = DID.loc[DID['Fluid'] == "Alcohol"]

var = 'TotalEvents'

sns.lmplot(x= var, y=  "Intake", data = data, hue = 'Sex', 
           ci = 68,
           height = figheight,
           hue_order=['Male','Female'],
           scatter_kws={'linewidths': 0, "s": 2.5},
           line_kws={"linewidth":0.5},
           legend = False
           )

plt.ylabel('Alcohol Intake (g/kg)', size = 5)
plt.xlabel('Drinking Events', size = 5)

plt.ylim([0,10])

plt.tick_params(labelsize=5, width=0.5, length = 2)

plt.legend(frameon = False, loc = 'lower right', fontsize = 5, markerscale = 2)

stat_out = pg.corr(data['Intake'], data[var], method='shepherd')
pval = sig_figs(stat_out['p-val'].iloc[0],4)
rval = sig_figs(stat_out['r'].iloc[0],4)

plt.text(x= 1, y = 9,  s = (f"r = {rval}\np = {pval}"), size = 5)

plt.tight_layout()
plt.savefig('Alcohol_Events.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

print(pval)
print(rval)

pg.ancova(data=data, dv='Intake', between='Sex', covar= var, effsize='np2').round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth

data = DID.loc[DID['Fluid'] == "Water"]

var = 'TotalEventDuration'

sns.lmplot(x= var, y=  "Intake", data = data, hue = 'Sex', 
           ci = 68,
           height = figheight,
           hue_order=['Male','Female'],
           scatter_kws={'linewidths': 0, "s": 2.5},
           line_kws={"linewidth":0.5},
           legend = False
           )

plt.ylabel('Water Intake (g/kg)', size = 5)
plt.xlabel('Total Event Duration (s)', size = 5)

plt.ylim([0,120])

plt.tick_params(labelsize=5, width=0.5, length = 2)

plt.legend(frameon = False, loc = 'lower right', fontsize = 5, markerscale = 2)

stat_out = pg.corr(data['Intake'], data[var], method='shepherd')
pval = sig_figs(stat_out['p-val'].iloc[0],4)
rval = sig_figs(stat_out['r'].iloc[0],4)

plt.text(x= 1, y = 110,  s = (f"r = {rval}\np = {pval}"), size = 5)

plt.tight_layout()
plt.savefig('Water_Event_Duration.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

print(pval)
print(rval)

pg.ancova(data=data, dv='Intake', between='Sex', covar= var, effsize='np2').round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth

data = DID.loc[DID['Fluid'] == "Alcohol"]

var = 'TotalEventDuration'

sns.lmplot(x= var, y=  "Intake", data = data, hue = 'Sex', 
           ci = 68,
           height = figheight,
           hue_order=['Male','Female'],
           scatter_kws={'linewidths': 0, "s": 2.5},
           line_kws={"linewidth":0.5},
           legend = False
           )

plt.ylabel('Alcohol Intake (g/kg)', size = 5)
plt.xlabel('Total Event Duration (s)', size = 5)

plt.ylim([0,10])

plt.tick_params(labelsize=5, width=0.5, length = 2)

plt.legend(frameon = False, loc = 'lower right', fontsize = 5, markerscale = 2)

stat_out = pg.corr(data['Intake'], data[var], method='shepherd')
pval = sig_figs(stat_out['p-val'].iloc[0],4)
rval = sig_figs(stat_out['r'].iloc[0],4)

plt.text(x= 1, y = 9,  s = (f"r = {rval}\np = {pval}"), size = 5)

plt.tight_layout()
plt.savefig('Alcohol_Event_Duration.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

print(pval)
print(rval)

pg.ancova(data=data, dv='Intake', between='Sex', covar= var, effsize='np2')

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth

data = DID.loc[DID['Fluid'] == "Alcohol"]

var = 'Drinks'

sns.lmplot(x= var, y=  "Intake", data = data, hue = 'Sex', 
           ci = 68,
           height = figheight,
           hue_order=['Male','Female'],
           scatter_kws={'linewidths': 0, "s": 2.5},
           line_kws={"linewidth":0.5},
           legend = False
           )

plt.ylabel('Alcohol Intake (g/kg)', size = 5)
plt.xlabel('Drinks', size = 5)

plt.ylim([0,10])

plt.tick_params(labelsize=5, width=0.5, length = 2)

plt.legend(frameon = False, loc = 'lower right', fontsize = 5, markerscale = 2)

stat_out = pg.corr(data['Intake'], data[var], method='shepherd')
pval = sig_figs(stat_out['p-val'].iloc[0],4)
rval = sig_figs(stat_out['r'].iloc[0],4)

plt.text(x= 1, y = 9,  s = (f"r = {rval}\np = {pval}"), size = 5)

plt.tight_layout()
plt.savefig('Alcohol_Drinks.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

print(pval)
print(rval)

pg.ancova(data=data, dv='Intake', between='Sex', covar= var, effsize='np2')

## Thirst State Correlations

### Does previous 24 hours of water intake correlate with water or alcohol intake during DID Task?

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/5
figheight = figwidth

data = data = DID.loc[DID['Fluid'] == "Water"]

print(pg.multivariate_normality(data[['Previous_Water', 'Intake']]))

h = sns.lmplot(x= 'Previous_Water', y=  "Intake", data = data, hue = 'Fluid',
           ci = 68,
           height = figheight,
           palette = ['#000000'],
           scatter_kws={'linewidths': 0, "s": 2.5},
           line_kws={"linewidth":0.5},
           legend = False
           )

h.set_axis_labels('Previous Inter-Session\nWater Intake (g/kg)', 'DID Session\nWater Intake (g/kg)', size = 5)

plt.ylim([0,120])

plt.yticks([0,20,40,60,80,100,120])

stat_out = pg.corr(data['Previous_Water'], data['Intake'], method='shepherd')
pval = stat_out['p-val'].iloc[0]
plt.text(x= 75, y = 100,  s = (f"r = {stat_out['r'].iloc[0].round(4)}\np = {pval.round(4)}"), size = 5)

plt.tick_params(labelsize=5, width=0.5, length = 2)

plt.tight_layout()
#plt.savefig('previouswaterdrinking.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.corr(data['Previous_Water'], data['Intake'], method='shepherd').round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/5
figheight = figwidth

data = DID.loc[DID['Fluid'] == "Water"]

print(pg.multivariate_normality(data[['Previous_Water', 'Intake']]))

h = sns.lmplot(x= 'Previous_Water', y=  "Intake", data = data, hue = 'Sex',
           ci = 68,
           height = figheight,
           hue_order=['Male','Female'],
           scatter_kws={'linewidths': 0, "s": 2.5},
           line_kws={"linewidth":0.5},
           legend = False
           )

h.set_axis_labels('Previous Inter-Session\nWater Intake (g/kg)', 'DID Session\nWater Intake (g/kg)', size = 5)

plt.ylim([0,120])
plt.yticks([0,20,40,60,80,100,120])

plt.tick_params(labelsize=5, width=0.5, length = 2)

plt.legend(frameon = False, loc = 'upper left', fontsize = 5, markerscale = 2)

plt.tight_layout()
plt.savefig('previouswaterLMbySex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.ancova(data=data, dv='Intake', between='Sex', covar='Previous_Water', effsize='np2').round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/5
figheight = figwidth

data = DID.loc[DID['Fluid'] == "Alcohol"]

print(pg.multivariate_normality(data[['Previous_Water', 'Intake']]))

h = sns.lmplot(x= 'Previous_Water', y=  "Intake", data = data, hue = 'Fluid',
           ci = 68,
           height = figheight,
           palette = ['#00a170'],
           scatter_kws={'linewidths': 0, "s": 2.5},
           line_kws={"linewidth":0.5},
           legend = False
           )

h.set_axis_labels('Previous Inter-Session\nWater Intake (g/kg)', 'DID Session\nAlcohol Intake (g/kg)', size = 5)

plt.ylim([0,10])

stat_out = pg.corr(data['Previous_Water'], data['Intake'], method='shepherd')
pval = stat_out['p-val'].iloc[0]
plt.text(x= 45, y = 8,  s = (f"r = {stat_out['r'].iloc[0].round(4)}\np = {pval.round(4)}"), size = 5)

plt.tick_params(labelsize=5, width=0.5, length = 2)

plt.tight_layout()
plt.savefig('previousalocholbyintersession.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.corr(data['Previous_Water'], data['Intake'], method='shepherd')

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/5
figheight = figwidth

data = DID.loc[DID['Fluid'] == "Alcohol"]

print(pg.multivariate_normality(data[['Previous_Water', 'Intake']]))

h = sns.lmplot(x= 'Previous_Water', y=  "Intake", data = data, hue = 'Sex',
           ci = 68,
           height = figheight,
           hue_order=['Male','Female'],
           scatter_kws={'linewidths': 0, "s": 2.5},
           line_kws={"linewidth":0.5},
           legend = False
           )

h.set_axis_labels('Previous Inter-Session\nWater Intake (g/kg)', 'DID Session\nAlcohol Intake (g/kg)', size = 5)

plt.ylim([0,10])

plt.tick_params(labelsize=5, width=0.5, length = 2)

plt.legend(frameon = False, loc = 'upper left', fontsize = 5, markerscale = 2)

plt.tight_layout()
plt.savefig('previouswaterALCOHOLbySex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.ancova(data=data, dv='Intake', between='Sex', covar='Previous_Water', effsize='np2').round(4)

In [ ]:
data = DID.loc[(DID['Fluid'] == "Alcohol") & (DID['Sex'] == "Female")]

print(pg.multivariate_normality(data[['Previous_Water', 'Intake']]))

sns.lmplot(x= 'Previous_Water', y=  "Intake", data = data, hue = 'Sex', ci = 68, height=8,
           scatter_kws={'linewidths': 0}, legend=False)

plt.ylim(0,10)

plt.xlabel("Previous Inter-Session\nWater Intake (g/kg)")
plt.ylabel("DID Session\nAlcohol Intake (g/kg)")

plt.tight_layout()

plt.show()
plt.clf()

pg.corr(data['Previous_Water'], data['Intake'], method='shepherd').round(4)

In [ ]:
data = DID.loc[(DID['Fluid'] == "Alcohol") & (DID['Sex'] == "Male")]

print(pg.multivariate_normality(data[['Previous_Water', 'Intake']]))

sns.lmplot(x= 'Previous_Water', y=  "Intake", data = data, hue = 'Sex', ci = 68, height=8, scatter_kws={'linewidth': 0}, legend=False)

plt.ylim(0,10)

plt.xlabel("Previous Inter-Session\nWater Intake (g/kg)")
plt.ylabel("DID Session\nAlcohol Intake (g/kg)")

plt.tight_layout()

plt.savefig('previouswaterALCOHOLbySex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.corr(data['Previous_Water'], data['Intake'], method='shepherd').round(4)

## Microstructure by Week

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Water")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'TotalEvents'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Events', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,500)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper right', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Water_Events_Time.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Alcohol")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'TotalEvents'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Alcohol Events', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,700)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper right', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Alcohol_Events_Time.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Water")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'TotalEventDuration'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Event Duration (s)', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,140)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper right', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Water_EventDuration_Week.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Alcohol")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'TotalEventDuration'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Alcohol Event Duration (s)', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,60)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Alcohol_EventDuration_Week.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pg.pairwise_tests(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data, padjust='sidak')

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Water")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'Drinks'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Drinks', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,200)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper right', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Water_Drinks_Week.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Alcohol")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'Drinks'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Alcohol Drinks', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,90)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Alcohol_Drinks_Week.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Water")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'Latency'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Latency to Drink (s)', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,6000)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper right', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Water_Latency_Week.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Alcohol")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'Latency'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Alcohol Latency to Drink (s)', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,9000)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Alcohol_Latency_Week.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Water")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'IDI'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Mean\nInter Drink Interval (s)', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,6000)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper right', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Water_IDI_Week.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Alcohol")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'IDI'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Alcohol Mean\nInter Drink Interval (s)', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,3000)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Alcohol_IDI_Week.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pg.pairwise_tests(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data, padjust='sidak')

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Water")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'TotalEvents30'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Water Events\nin First 30 Minutes', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,70)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper right', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Water_Events_30.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/6
figheight = figwidth*1.6
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = DID.loc[(DID['DIDWeek'] > 0) & (DID['Fluid'] == "Alcohol")]
data['DIDWeek'] = data['DIDWeek'].astype('int')

var = 'TotalEvents30'

sns.stripplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", jitter = True, edgecolor='gray', alpha =0.5, hue_order=['Male','Female'], size = 1)
sns.pointplot(data = data, x = 'DIDWeek', y = var, hue = "Sex", errorbar='se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5)

plt.ylabel('Alcohol Events\nin First 30 Minutes', size = 5)
plt.xlabel('Drinking Week', size = 5)

plt.ylim(0,100)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:2], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('Alcohol_Events_30.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, between='Sex', within='DIDWeek', subject='ID', data=data).round(4)

# FP DATALOADER

In [ ]:
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from scipy import signal
from sklearn.linear_model import RANSACRegressor

#define biexponential function
def _2exponential(x, a, b, c, d,):
    return a*np.exp(b*x) + c*np.exp(d*x)

In [ ]:
FP = pd.DataFrame()

for i in DID['ID'].unique():
    print("Initalizing...", i)
    files = []
    for j in DID.loc[DID['ID'] == i, "Session"].unique():
        wd = os.path.join(os.getcwd(), "Session" + " " + str(j))
        current_session = os.path.join(wd, str(i))
        filename = glob.glob(current_session+"/"+"all_raw*.csv")
        if len(filename) != 0:
            files.append(filename)
            print("Adding Session...", j)
            for k in files:
                temp_df = pd.read_csv(k[0])
                temp_df['ID'] = i
                temp_df['Session'] = j
                temp_df['Date'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Date'].iloc[0]
                temp_df['BottleSide'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'BottleSide'].iloc[0]
                temp_df['Fluid'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Fluid'].iloc[0]
                temp_df['SessionLength'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'SessionLength'].iloc[0]
                temp_df['DIDWeek'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'DIDWeek'].iloc[0]
                temp_df['Sex'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Sex'].iloc[0]
            FP = pd.concat([FP, temp_df])

FP.to_parquet('FP.parquet')

In [ ]:
for animal in FP['ID'].unique():
    print("Initalizing...", animal)

    for session in FP.loc[FP['ID'] == animal, "Session"].unique():
        print(session)
        
        FPSession = FP.loc[(FP['ID'] == animal) & (FP['Session'] == session)].copy()

        FPSession = FPSession.loc[FPSession['FrameCounter'] > 300]

        ### Fit Left Isosbestic
        try:
            popt_left, pcov_left = curve_fit(_2exponential,
                                np.array(FPSession.loc[FPSession['LedState'] == 1, 'Timestamp']),
                                np.array(FPSession.loc[FPSession['LedState'] == 1, 'Region2G']),
                                p0=[0,-0.0025,0.000025,0], maxfev=100000000
                                )
        except ValueError:
            print(animal, session, "Fit Failed Left Side")

        FPSession.loc[FPSession['LedState'] == 1, 'IsosbesticFit_Left'] = _2exponential(np.array(FPSession.loc[FPSession['LedState'] == 1, 'Timestamp']), *popt_left) #write fit y values to array

        ### Fit Right Isosbestic
        try:
            popt_right, pcov_right = curve_fit(_2exponential,
                                np.array(FPSession.loc[FPSession['LedState'] == 1, 'Timestamp']),
                                np.array(FPSession.loc[FPSession['LedState'] == 1, 'Region3G']),
                                p0=[0,-0.0025,0.000025,0], maxfev=100000000
                                )
        except ValueError:
            print(animal, session, "Fit Failed Left Side")

        FPSession.loc[FPSession['LedState'] == 1, 'IsosbesticFit_Right'] = _2exponential(np.array(FPSession.loc[FPSession['LedState'] == 1, 'Timestamp']), *popt_right) #write fit y values to array

        ###Scale Left Fit

        X = np.array(FPSession.loc[FPSession['LedState'] == 1, 'IsosbesticFit_Left'])
        y = np.array(FPSession.loc[FPSession['LedState'] == 2, 'Region2G'])

        if len(X) != len(y):
            X = signal.resample(X, len(y))

        X = X.reshape(-1,1)
        y = y.reshape(-1,1)

        robust_linear_model = RANSACRegressor(random_state=42)
        robust_linear_model.fit(X.reshape(-1,1), y.reshape(-1,1))
        FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region2G"] = robust_linear_model.predict(X)

        ###Scale Right Fit

        X = np.array(FPSession.loc[FPSession['LedState'] == 1, 'IsosbesticFit_Right']).reshape(-1, 1)
        y = np.array(FPSession.loc[FPSession['LedState'] == 2, 'Region3G']).reshape(-1, 1)

        if len(X) != len(y):
            X = signal.resample(X, len(y))

        X = X.reshape(-1,1)
        y = y.reshape(-1,1)

        robust_linear_model = RANSACRegressor(random_state=42)
        robust_linear_model.fit(X, y)
        FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region3G"] = robust_linear_model.predict(X)

        ### Lowpass Left Signal
        FPSession.loc[FPSession['LedState'] == 2, 'lowpass470_left'] = signal.sosfilt(signal.butter(2, 6, fs = 30, btype='lowpass', output = 'sos'), FPSession.loc[FPSession['LedState'] == 2, "Region2G"])

        ### Lowpass Right Signal
        FPSession.loc[FPSession['LedState'] == 2, 'lowpass470_right'] = signal.sosfilt(signal.butter(2, 6, fs = 30, btype='lowpass', output = 'sos'), FPSession.loc[FPSession['LedState'] == 2, "Region3G"])

        ### Left Graphs
        #plot raw 415
        plt.plot(FPSession.loc[FPSession['LedState'] == 1, "Timestamp"], FPSession.loc[FPSession['LedState'] == 1, "Region2G"], 'b-', label='Raw 415nm Channel')
        #plot isobestic fit
        plt.plot(FPSession.loc[FPSession['LedState'] == 1, "Timestamp"], FPSession.loc[FPSession['LedState'] == 1, "IsosbesticFit_Left"], 'r-', label='Isobestic Fit')
        #plot raw 470
        plt.plot(FPSession.loc[FPSession['LedState'] == 2, "Timestamp"], FPSession.loc[FPSession['LedState'] == 2, "Region2G"], 'g-', label='Raw 470nm Channel')
        #plot lowpass filtered 470 data
        plt.plot(FPSession.loc[FPSession['LedState'] == 2, "Timestamp"], FPSession.loc[FPSession['LedState'] == 2, "lowpass470_left"], 'c-', label='6 Hz Lowpass Filtered 470nm Channel')
        #plot scaled isobestic fit
        plt.plot(FPSession.loc[FPSession['LedState'] == 2, "Timestamp"], FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region2G"], 'y-', label='Scaled Isobestic Fit')

        plt.tight_layout()
        plt.title("Left DLS")
        plt.xlabel('Time (s)')
        plt.ylabel('AU')
        plt.legend()
        plt.show()

        ### Right Graphs
        #plot raw 415
        plt.plot(FPSession.loc[FPSession['LedState'] == 1, "Timestamp"], FPSession.loc[FPSession['LedState'] == 1, "Region3G"], 'b-', label='Raw 415nm Channel')
        #plot isobestic fit
        plt.plot(FPSession.loc[FPSession['LedState'] == 1, "Timestamp"], FPSession.loc[FPSession['LedState'] == 1, "IsosbesticFit_Right"], 'r-', label='Isobestic Fit')
        #plot raw 470
        plt.plot(FPSession.loc[FPSession['LedState'] == 2, "Timestamp"], FPSession.loc[FPSession['LedState'] == 2, "Region3G"], 'g-', label='Raw 470nm Channel')
        #plot lowpass filtered 470 data
        plt.plot(FPSession.loc[FPSession['LedState'] == 2, "Timestamp"], FPSession.loc[FPSession['LedState'] == 2, "lowpass470_right"], 'c-', label='6 Hz Lowpass Filtered 470nm Channel')
        #plot scaled isobestic fit
        plt.plot(FPSession.loc[FPSession['LedState'] == 2, "Timestamp"], FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region3G"], 'y-', label='Scaled Isobestic Fit')

        plt.tight_layout()
        plt.title("Right DLS")
        plt.xlabel('Time (s)')
        plt.ylabel('AU')
        plt.legend()
        plt.show()

        #Calculate dF/F
        FP.loc[(FP['ID'] == animal) & (FP['Session'] == session) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), "LeftSignal_dF"] = ((FPSession.loc[FPSession['LedState'] == 2, "lowpass470_left"]) / (FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region2G"]))
        FP.loc[(FP['ID'] == animal) & (FP['Session'] == session) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), "LeftSignal_dFF"] = (((FPSession.loc[FPSession['LedState'] == 2, "lowpass470_left"])-(FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region2G"]))/(FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region2G"]))
        FP.loc[(FP['ID'] == animal) & (FP['Session'] == session) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), "LeftSignal_dFF_zscore"] = stats.zscore((((FPSession.loc[FPSession['LedState'] == 2, "lowpass470_left"])-(FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region2G"]))/(FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region2G"])), nan_policy='omit')

        
        FP.loc[(FP['ID'] == animal) & (FP['Session'] == session) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), "RightSignal_dF"] = ((FPSession.loc[FPSession['LedState'] == 2, "lowpass470_right"]) / (FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region3G"]))
        FP.loc[(FP['ID'] == animal) & (FP['Session'] == session) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), "RightSignal_dFF"] = (((FPSession.loc[FPSession['LedState'] == 2, "lowpass470_right"])-(FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region3G"]))/(FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region3G"]))
        FP.loc[(FP['ID'] == animal) & (FP['Session'] == session) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), "RightSignal_dFF_zscore"] = stats.zscore((((FPSession.loc[FPSession['LedState'] == 2, "lowpass470_right"])-(FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region3G"]))/(FPSession.loc[FPSession['LedState'] == 2, "Scaled_Isobestic_Fit_Region3G"])), nan_policy='omit')

In [ ]:
#remove data because the cable was broke, bad recording, or nonsense fit
FP.loc[(FP['ID'] == 'P1') & (FP['Session'] == 5) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P1') & (FP['Session'] == 9) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P1') & (FP['Session'] == 13) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P1') & (FP['Session'] == 17) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P1') & (FP['Session'] == 17) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P1') & (FP['Session'] == 21) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan

FP.loc[(FP['ID'] == 'P2') & (FP['Session'] == 4) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P2') & (FP['Session'] == 4) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P2') & (FP['Session'] == 8) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P2') & (FP['Session'] == 12) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P2') & (FP['Session'] == 20) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan

FP.loc[(FP['ID'] == 'P3') & (FP['Session'] == 3) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P3') & (FP['Session'] == 7) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P3') & (FP['Session'] == 11) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P3') & (FP['Session'] == 15) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P3') & (FP['Session'] == 23) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P3') & (FP['Session'] == 23) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P3') & (FP['Session'] == 27) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P3') & (FP['Session'] == 27) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan

FP.loc[(FP['ID'] == 'P4') & (FP['Session'] == 2) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P4') & (FP['Session'] == 6) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P4') & (FP['Session'] == 10) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P4') & (FP['Session'] == 14) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P4') & (FP['Session'] == 14) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P4') & (FP['Session'] == 18) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P4') & (FP['Session'] == 30) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan

FP.loc[(FP['ID'] == 'P5') & (FP['Session'] == 2) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan

FP.loc[(FP['ID'] == 'P8') & (FP['Session'] == 4) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P8') & (FP['Session'] == 25) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan

FP.loc[(FP['ID'] == 'P10') & (FP['Session'] == 27) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P10') & (FP['Session'] == 27) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan

FP.loc[(FP['ID'] == 'P12') & (FP['Session'] == 9) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P12') & (FP['Session'] == 13) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P12') & (FP['Session'] == 30) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan

FP.loc[(FP['ID'] == 'P14') & (FP['Session'] == 3) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'LeftSignal_dFF_zscore'] = np.nan
FP.loc[(FP['ID'] == 'P14') & (FP['Session'] == 3) & (FP['LedState'] == 2) & (FP['FrameCounter'] > 300), 'RightSignal_dFF_zscore'] = np.nan

#create empty dataframe named timestamped_FP
timestamped_FP = pd.DataFrame()

for i in FP['ID'].unique():
    print("Initalizing...", i)
    #create an empty list for storing filenames
    files = []
    
    for j in FP.loc[FP['ID'] == i, "Session"].unique():
        wd = os.path.join(os.getcwd(), "Session" + " " + str(j))
        current_session = os.path.join(wd, str(i))

        filename = glob.glob(current_session+"/"+"FP_Timestamps*.csv")

        if len(filename) != 0:
            files.append(filename)
            print("Adding Session...", j)

            for k in files:
                temp_df = pd.read_csv(k[0], header=None)
                temp_df.columns = ('SYNC_Time','FrameCounter')
                temp_df = FP.loc[(FP['ID'] == i) & (FP['Session'] == j)].merge(temp_df, left_on='FrameCounter', right_on='FrameCounter')
               
            timestamped_FP = pd.concat([timestamped_FP, temp_df])

#lets do a little clean up here before we move on to adding licks
#drop unnecessary columns
timestamped_FP = timestamped_FP.drop(['Region0R', 'Region1R', 'Stimulation', 'Output0', 'Output1', 'Input0','Input1', 'Region4R', 'Region5G'], axis=1)

#create empty dataframe named leftlicks_timestamped_FP to assign left licks to sync time 
leftlicks_timestamped_FP = pd.DataFrame()

for i in timestamped_FP['ID'].unique():
    print("Initalizing...", i)
    files = []
    
    for j in timestamped_FP.loc[timestamped_FP['ID'] == i, "Session"].unique():
        wd = os.path.join(os.getcwd(), "Session" + " " + str(j))
        current_session = os.path.join(wd, str(i))

        filename = glob.glob(current_session+"/"+"left_licks*.csv")

        if len(filename) != 0:
            files.append(filename)
            print("Adding Session...", j)
            
            for k in files:
                temp_df = pd.read_csv(k[0], header=None)             
                temp_df.columns = ('SYNC_Time','LeftLick')
                temp_df['ID'] = i
                temp_df['Session'] = j
                temp_df['Date'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Date'].iloc[0]
                temp_df['BottleSide'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'BottleSide'].iloc[0]
                temp_df['Fluid'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Fluid'].iloc[0]
                temp_df['SessionLength'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'SessionLength'].iloc[0]
                temp_df['DIDWeek'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'DIDWeek'].iloc[0]
                temp_df['Sex'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Sex'].iloc[0]
                temp_df = pd.concat([timestamped_FP.loc[(timestamped_FP['ID'] == i) & (timestamped_FP['Session'] == j)], temp_df]).sort_values(by='SYNC_Time')

            #add temp_df to leftlicks_timestamped_FP
            leftlicks_timestamped_FP = pd.concat([leftlicks_timestamped_FP, temp_df])

#create empty dataframe named rightleftlicks_timestamped_FP to assign right licks to sync time after leftlicks have already been accomplished 
rightleftlicks_timestamped_FP = pd.DataFrame()

for i in timestamped_FP['ID'].unique():
    print("Initalizing...", i)
    files = []
    
    for j in timestamped_FP.loc[timestamped_FP['ID'] == i, "Session"].unique():
        wd = os.path.join(os.getcwd(), "Session" + " " + str(j))
        current_session = os.path.join(wd, str(i))

        filename = glob.glob(current_session+"/"+"right_licks*.csv")

        if len(filename) != 0:
            files.append(filename)
            print("Adding Session...", j)
            
            for k in files:
                temp_df = pd.read_csv(k[0], header=None)             
                temp_df.columns = ('SYNC_Time','RightLick')
                temp_df['ID'] = i
                temp_df['Session'] = j
                temp_df['Date'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Date'].iloc[0]
                temp_df['BottleSide'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'BottleSide'].iloc[0]
                temp_df['Fluid'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Fluid'].iloc[0]
                temp_df['SessionLength'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'SessionLength'].iloc[0]
                temp_df['DIDWeek'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'DIDWeek'].iloc[0]
                temp_df['Sex'] = DID.loc[(DID['ID'] == i) & (DID['Session'] == j), 'Sex'].iloc[0]
                temp_df = pd.concat([leftlicks_timestamped_FP.loc[(leftlicks_timestamped_FP['ID'] == i) & (leftlicks_timestamped_FP['Session'] == j)], temp_df]).sort_values(by='SYNC_Time')

            #add temp_df to leftlicks_timestamped_FP
            rightleftlicks_timestamped_FP = pd.concat([rightleftlicks_timestamped_FP, temp_df])

# rename rightlick_leftlick_timestamped_FP to FP_FINAL
FP_FINAL = rightleftlicks_timestamped_FP  

#if bottleside is left and leftlick is Ture, then create a new column called Lick and set it to True
FP_FINAL.loc[(FP_FINAL['BottleSide'] == 'Left') & (FP_FINAL['LeftLick'] == True), 'Lick'] = True
FP_FINAL.loc[(FP_FINAL['BottleSide'] == 'Left') & (FP_FINAL['LeftLick'] == False), 'Lick'] = False
#if bottleside is right and rightlick is Ture, then create a new column called Lick and set it to True
FP_FINAL.loc[(FP_FINAL['BottleSide'] == 'Right') & (FP_FINAL['RightLick'] == True), 'Lick'] = True
FP_FINAL.loc[(FP_FINAL['BottleSide'] == 'Right') & (FP_FINAL['RightLick'] == False), 'Lick'] = False

#align licks to bout start
for i in FP_FINAL['ID'].unique():
    print("Initalizing...", i)
    for j in FP_FINAL.loc[FP_FINAL['ID'] == i, "Session"].unique():
        print("Session", j)
        if len(FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'] == True)]) != 0:
            #set some vars 
            windowsize = 10000 #ms
            drinkstart = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'] == True), "SYNC_Time"].iloc[0]
            windowstart = 0
            windowend = 0
            drinknumber = 1
            while len(FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'] == True) & (FP_FINAL['SYNC_Time'] > drinkstart), 'SYNC_Time']) > 0:
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['SYNC_Time'] == drinkstart), "DrinkStart"] = "True"
                windowstart = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['SYNC_Time'] >= drinkstart - (windowsize/2)), "SYNC_Time"].min()
                windowend = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['SYNC_Time'] <= drinkstart + (windowsize/2)), "SYNC_Time"].max()
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['SYNC_Time'] >= windowstart) & (FP_FINAL['SYNC_Time'] <= windowend), 'DrinkWindow'] = True
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['SYNC_Time'] >= windowstart) & (FP_FINAL['SYNC_Time'] <= windowend), 'DrinkNumber'] = drinknumber
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['SYNC_Time'] >= drinkstart) & (FP_FINAL['SYNC_Time'] <= windowend), "TimeFromDrink"] = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['SYNC_Time'] >= drinkstart) & (FP_FINAL['SYNC_Time'] <= windowend), 'SYNC_Time'] - drinkstart
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['SYNC_Time'] >= windowstart) & (FP_FINAL['SYNC_Time'] <= drinkstart), 'TimeFromDrink'] = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['SYNC_Time'] >= windowstart) & (FP_FINAL['SYNC_Time'] <= drinkstart), 'SYNC_Time'] - drinkstart
                drinknumber += 1
                print('Found Drink Number ...', drinknumber)
                
                try:
                    drinkstart = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) &  (FP_FINAL['Lick'] == True) & (FP_FINAL['SYNC_Time'] > windowend + (windowsize/2)),'SYNC_Time'].iloc[0]
                except IndexError:
                    break
            
            print('Analyzing...')
            for k in FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['LedState'] == 2), "DrinkNumber"].unique():
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkNumber'] == k), 'LeftSignal_dFF_zscore_filtered'] = gaussian_filter1d(np.array(FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkNumber'] == k), 'LeftSignal_dFF_zscore']), 2)
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkNumber'] == k), 'RightSignal_dFF_zscore_filtered'] = gaussian_filter1d(np.array(FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkNumber'] == k), 'RightSignal_dFF_zscore']), 2)
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkNumber'] == k), 'DrinkLength'] = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].notnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['DrinkNumber'] == k), 'TimeFromDrink'].max()/1000
                

for i in FP_FINAL['ID'].unique():
    print(i)
    for j in FP_FINAL.loc[FP_FINAL['ID'] == i, "Session"].unique():
        print(j)
        for k in FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j), "DrinkNumber"].dropna().unique():
            print(k)
            if len(FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['DrinkNumber'] == k)]) == 150:
                   FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['DrinkNumber'] == k), 'TimeFromDrinkRound'] = np.linspace(-5, 5, num=150)
            if len(FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['DrinkNumber'] == k)]) == 149:
                time_of_first_frame = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['DrinkNumber'] == k),'FrameCounter'].min()
                #go grab one more frame...set it to the current drink number...and assign the drinkwindow to be true....
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['FrameCounter'] >= time_of_first_frame - 2) & (FP_FINAL['FrameCounter'] < time_of_first_frame), 'DrinkNumber'] = k
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['FrameCounter'] >= time_of_first_frame - 2) & (FP_FINAL['FrameCounter'] < time_of_first_frame), 'DrinkWindow'] = True
                
                #refilter with new frame...
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkNumber'] == k), 'LeftSignal_dFF_zscore_filtered'] = gaussian_filter1d(np.array(FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkNumber'] == k), 'LeftSignal_dFF_zscore']), 2)
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkNumber'] == k), 'RightSignal_dFF_zscore_filtered'] = gaussian_filter1d(np.array(FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['Lick'].isnull()) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkNumber'] == k), 'RightSignal_dFF_zscore']), 2)

                #then assign linspace...
                FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['LedState'] == 2) & (FP_FINAL['DrinkWindow'] == True) & (FP_FINAL['DrinkNumber'] == k), 'TimeFromDrinkRound'] = np.linspace(-5, 5, num=150)

FP_FINAL.to_parquet('FP_FINAL_THIS_ONE_IS_THE_SMOOTH_ONE.parquet')

# FP FINAL ANALYSIS

In [5]:
DID = pd.read_parquet('DID.parquet')
masterdrinker = pd.read_excel('masterdrinker.parquet')
FP_FINAL = pd.read_parquet('FP_Final.parquet')

## Figure 3 - Supplement 1

### Left

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water")].copy()

#badrecordingfromchewedcable
data.loc[(data['ID'] == "P9") & (data['Session'] == 7), var] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), var] = np.nan

data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,  data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = "Sex",
             hue_order= ['Male','Female']
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:2] + handles[3:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Water_BySex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, within='TimeFromDrinkRound', between='Sex', subject='ID', data=data)

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water")].copy()

#badrecordingfromchewedcable
data.loc[(data['ID'] == "P9") & (data['Session'] == 7), var] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), var] = np.nan
data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-5,20)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
#plt.savefig('Left_Water_BySex_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))


In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water")].copy()

#badrecordingfromchewedcable
data.loc[(data['ID'] == "P9") & (data['Session'] == 7), var] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), var] = np.nan
data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Left_Water_BySex_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))


In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water")].copy()

#badrecordingfromchewedcable
data.loc[(data['ID'] == "P9") & (data['Session'] == 7), var] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), var] = np.nan
data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Left_Water_BySex_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))

### Right

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water")].copy()

#badrecording
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), var] = np.nan

data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,  data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = "Sex",
             hue_order= ['Male','Female']
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:2] + handles[3:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Water_BySex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, within='TimeFromDrinkRound', between='Sex', subject='ID', data=data)

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water")].copy()

#badrecording
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), var] = np.nan
data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-5,20)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Right_Water_BySex_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water")].copy()

#badrecording
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), var] = np.nan
data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Right_Water_BySex_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water")].copy()

#badrecording
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), var] = np.nan
data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Right_Water_BySex_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))


## Figure 4

### Males

#### Left

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#badrecording
data.loc[(data['ID'] == "P9") & (data['Session'] == 7), var] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'

sns.lineplot(x = "TimeFromDrinkRound", y = var, data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = 'Week', hue_order = ['Week 1','Week 2','Week 3'],
             palette=['#262730','#8DA7BE','#DD6E42']
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Water_Male_ByWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv= var, within=['DIDWeek','TimeFromDrinkRound'], subject = 'ID', data = data, detailed=True).round(4)

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#badrecording
data.loc[(data['ID'] == "P9") & (data['Session'] == 7), var] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Water_Male_ByWeek_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#badrecording
data.loc[(data['ID'] == "P9") & (data['Session'] == 7), var] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)


plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Water_Male_ByWeek_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#badrecording
data.loc[(data['ID'] == "P9") & (data['Session'] == 7), var] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Water_Male_ByWeek_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

#### Right

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#bad recordings
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'

sns.lineplot(x = "TimeFromDrinkRound", y = var, data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = 'Week', hue_order = ['Week 1','Week 2','Week 3'],
             palette=['#262730','#8DA7BE','#DD6E42']
            )

plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Water_Male_ByWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv= var, within=['TimeFromDrinkRound','DIDWeek'], subject = 'ID', data = data, detailed=True)

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#bad recordings
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Water_Male_ByWeek_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#bad recordings
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)


plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Water_Male_ByWeek_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#bad recordings
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Water_Male_ByWeek_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

### Females

#### Left

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'

sns.lineplot(x = "TimeFromDrinkRound", y = var, data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = 'Week', hue_order = ['Week 1','Week 2','Week 3'],
             palette=['#262730','#8DA7BE','#DD6E42']
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Water_FEMale_ByWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv= var, within=['DIDWeek','TimeFromDrinkRound'], subject = 'ID', data = data, detailed=True)

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Female")].copy()


data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Water_FEMale_ByWeek_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)


plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Water_FEMale_ByWeek_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Water_FEMale_ByWeek_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

#### Right

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'

sns.lineplot(x = "TimeFromDrinkRound", y = var, data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = 'Week', hue_order = ['Week 1','Week 2','Week 3'],
             palette=['#262730','#8DA7BE','#DD6E42']
            )

plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Water_FEMale_ByWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv= var, within=['TimeFromDrinkRound','DIDWeek'], subject = 'ID', data = data, detailed=True)

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Water_FEMale_ByWeek_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)


plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Water_FEMale_ByWeek_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Water') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 1','Week 2','Week 3'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Water_FEMale_ByWeek_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

## Figure 3 - Supplement 2

### Left

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol")].copy()

#badrecordings
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), var] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), var] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), var] = np.nan

data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,  data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = "Sex",
             hue_order= ['Male','Female']
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:2] + handles[3:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Alcohol_BySex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, within='TimeFromDrinkRound', between='Sex', subject='ID', data=data)

In [ ]:
posthoc = pg.pairwise_tests(dv=var, within='TimeFromDrinkRound', between='Sex', subject='ID', data=data)
posthoc.loc[(posthoc['Contrast'] == "TimeFromDrinkRound * Sex") & (posthoc['p-unc'] < 0.05)]

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol")].copy()

#badrecordings
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), var] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), var] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-5,20)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Left_Alcohol_BySex_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))


In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol")].copy()

#badrecordings
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), var] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), var] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Left_Alcohol_BySex_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))


In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water")].copy()

#badrecordings
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), var] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), var] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Left_Alcohol_BySex_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))

### Right

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol")].copy()

#badrecordings
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), var] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,  data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = "Sex",
             hue_order= ['Male','Female']
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:2] + handles[3:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Alcohol_BySex.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.mixed_anova(dv=var, within='TimeFromDrinkRound', between='Sex', subject='ID', data=data)

In [ ]:
posthoc = pg.pairwise_tests(dv=var, within='TimeFromDrinkRound', between='Sex', subject='ID', data=data)
posthoc.loc[(posthoc['Contrast'] == "TimeFromDrinkRound * Sex") & (posthoc['p-unc'] < 0.05)]

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol")].copy()

#badrecordings
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), var] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-5,20)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Right_Alcohol_BySex_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol")].copy()

#badrecordings
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), var] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), var] = np.nan
data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Right_Alcohol_BySex_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol")].copy()

#badrecordings
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), var] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), var] = np.nan
data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'


for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Sex", ascending = False)

sns.stripplot(data = data, x = 'Sex', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= False, jitter=True,)
sns.pointplot(data = data, x = 'Sex', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[0:0], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Right_Alcohol_BySex_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

normalality = pg.normality(data, dv=var, group='Sex', method="shapiro")
if False in normalality['normal'].unique():
    print(pg.mwu(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], alternative='two-sided'))
else:
    print(pg.ttest(x = data.loc[data['Sex'] == "Male", var], y = data.loc[data['Sex'] == "Female", var], correction='auto'))


## Figure 5

### Males

#### Left

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Male")].copy()

data.loc[(data['ID'] == "P8") & (data['Session'] == 16), var] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), var] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

sns.lineplot(x = "TimeFromDrinkRound", y = var, data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = 'Week', hue_order = ['Week 4','Week 5','Week 6'],
             palette=['#262730','#8DA7BE','#DD6E42']
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Alcohol_Male_ByWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv= var, within=['TimeFromDrinkRound','DIDWeek'], subject = 'ID', data = data, detailed=True)

In [ ]:
posthoc = pg.pairwise_tests(dv= var, within=['TimeFromDrinkRound','DIDWeek'], subject = 'ID', data = data, padjust='fdr')
posthoc.loc[(posthoc['Contrast'] == "TimeFromDrinkRound * DIDWeek") & (posthoc['p-corr'] < 0.05)].sort_values(by=['TimeFromDrinkRound'])

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#badrecording
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), var] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), var] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Alcohol_Male_ByWeek_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#badrecording
#badrecording
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), var] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), var] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), var] = np.nan


data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)


plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Alcohol_Male_ByWeek_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#badrecording
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), var] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), var] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), var] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), var] = np.nan


data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Alcohol_Male_ByWeek_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

#### Right

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#bad recordings
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

sns.lineplot(x = "TimeFromDrinkRound", y = var, data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = 'Week', hue_order = ['Week 4','Week 5','Week 6'],
             palette=['#262730','#8DA7BE','#DD6E42']
            )

plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Alcohol_Male_ByWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv= var, within=['TimeFromDrinkRound','DIDWeek'], subject = 'ID', data = data, detailed=True)

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#bad recordings
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Alcohol_Male_ByWeek_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#bad recordings
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)


plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Alcohol_Male_ByWeek_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Male")].copy()

#bad recordings
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Alcohol_Male_ByWeek_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

### Females

#### Left

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

sns.lineplot(x = "TimeFromDrinkRound", y = var, data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = 'Week', hue_order = ['Week 4','Week 5','Week 6'],
             palette=['#262730','#8DA7BE','#DD6E42']
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Alcohol_Female_ByWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv= var, within=['TimeFromDrinkRound','DIDWeek'], subject = 'ID', data = data, detailed=True)

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Alcohol_FEMale_ByWeek_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)


plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Alcohol_FEMale_ByWeek_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "LeftSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Female")].copy()

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Left AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left_Alcohol_FEMale_ByWeek_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

#### Right

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Female")].copy()

#bad recordings
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

sns.lineplot(x = "TimeFromDrinkRound", y = var, data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = 'Week', hue_order = ['Week 4','Week 5','Week 6'],
             palette=['#262730','#8DA7BE','#DD6E42']
            )

plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Alcohol_FEMale_ByWeek.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv= var, within=['TimeFromDrinkRound','DIDWeek'], subject = 'ID', data = data, detailed=True)

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Female")].copy()

#bad recordings
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Alcohol_FEMale_ByWeek_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Female")].copy()

#bad recordings
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)


plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Alcohol_FEMale_ByWeek_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

In [ ]:
var = "RightSignal_dFF_zscore_filtered"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == 'Alcohol') &
                    (FP_FINAL['Sex'] == "Female")].copy()

#bad recordings
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), var] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 1, 'Week'] = 'Week 1'
data.loc[data['DIDWeek'] == 2, 'Week'] = 'Week 2'
data.loc[data['DIDWeek'] == 3, 'Week'] = 'Week 3'
data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Week', y = var, hue = "Week", hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], alpha = 0.5, size = 1,  dodge= False, jitter=True)
sns.pointplot(data = data, x = 'Week', y = var, hue = "Week", errorbar = 'se', hue_order = ['Week 4','Week 5','Week 6'], palette=['#262730','#8DA7BE','#DD6E42'], scale = 0.5, errwidth = 0.5, dodge=False, join = False)

plt.ylabel("Right AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)
plt.xticks(rotation = 25)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[0:0])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right_Alcohol_FEMale_ByWeek_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

variance = pg.homoscedasticity(data=data, dv=var, group='Week')
if variance['equal_var'].iloc[0] == False:
    print("Welch")
    print(pg.welch_anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_gameshowell(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['pval'] < 0.05])
else:
    print("ANOVA")
    print(pg.anova(dv = var, between='Week', data = data))
    posthoc = pg.pairwise_tukey(dv = var, between='Week', data = data)
    print(posthoc.loc[posthoc['p-tukey'] < 0.05])

## Figure 3

In [ ]:
# by sex
pagewidth = 180/25.4
figwidth = pagewidth*0.75
figheight = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()
handles = []

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan

#combine data columns into one column
data['Sex-Fluid'] = data['Sex'] + ' - ' + data["Fluid"]
data = pd.melt(data, id_vars=['ID','Session','DrinkNumber','Sex','Fluid','Sex-Fluid','TimeFromDrinkRound'], value_vars=['RightSignal_dFF_zscore_filtered','LeftSignal_dFF_zscore_filtered'])
data['Sex-Fluid-Side'] = data['Sex-Fluid'] + '-' + data['variable']
data = data.reset_index()

plt.hlines(y = 0 , xmin = -5, xmax = 5, linestyles = 'dashed', color = 'black', alpha = 0.5, linewidth = 0.5)
plt.vlines(x = 0 , ymin = -0.95, ymax = 1.45, linestyles = 'dashed', color = 'black', alpha = 0.5, linewidth = 0.5)

sns.lineplot(x = "TimeFromDrinkRound", y = "value", 
             hue = 'Sex-Fluid',
             style = 'variable',
             data = data,
             hue_order = ['Male - Water','Female - Water','Male - Alcohol', 'Female - Alcohol'],
             style_order=['LeftSignal_dFF_zscore_filtered','RightSignal_dFF_zscore_filtered'],
             estimator=np.mean, errorbar='se', size = 1)

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("zscore ΔF/F", size = 5)
plt.xlabel("Time from Bout Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('RightVLeft.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv='value', between=['Sex','Fluid','TimeFromDrinkRound','variable'], data = data, detailed=True)

In [ ]:
pd.options.display.max_rows = 999

posthoc = pg.pairwise_tests(dv='value', between=['TimeFromDrinkRound',"Sex-Fluid"], data = data, padjust='sidak')
posthoc.loc[(posthoc['Contrast'] == "TimeFromDrinkRound * Sex-Fluid") & (posthoc['p-corr'] < 0.05)].sort_values(by = 'TimeFromDrinkRound')

In [ ]:
posthoc = pg.pairwise_tests(dv='value', between=['TimeFromDrinkRound',"variable"], data = data, padjust='sidak')
posthoc.loc[(posthoc['Contrast'] == "TimeFromDrinkRound * variable") & (posthoc['p-corr'] < 0.05)].sort_values(by = 'TimeFromDrinkRound')

In [ ]:
# by sex
pagewidth = 180/25.4
figwidth = pagewidth*0.25
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan

#combine data columns into one column
data['Sex-Fluid'] = data['Sex'] + ' - ' + data["Fluid"]
data = pd.melt(data, id_vars=['ID','Session','DrinkNumber','Sex','Fluid','Sex-Fluid','TimeFromDrinkRound'], value_vars=['LeftSignal_dFF_zscore_filtered','RightSignal_dFF_zscore_filtered'])
data = data.groupby(['ID','Session','Sex','Fluid','Sex-Fluid','DrinkNumber','variable']).max()
data = data.reset_index()

sns.stripplot(y = "value", x = 'variable', hue = 'Sex-Fluid', data = data, legend = False, hue_order = ['Male - Water','Female - Water','Male - Alcohol', 'Female - Alcohol'], alpha = 0.5, size = 1,  dodge= True, jitter=True)
sns.pointplot(y = "value", x = "variable", hue = 'Sex-Fluid', dodge=.6, data = data, estimator=np.mean, errorbar='se', hue_order = ['Male - Water','Female - Water','Male - Alcohol', 'Female - Alcohol'], scale = 0.5, errwidth = 0.5, join = False)

plt.ylabel("Max zscore ΔF/F", size = 5)
plt.xlabel(" ")

plt.ylim(-5,20)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[:], frameon=False, loc = 'upper right', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('BYside_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv='value', between=['variable','Sex-Fluid'], data = data, detailed=True)

In [ ]:
posthoc = pg.pairwise_tests(dv='value', between=['variable','Sex-Fluid'], data = data, padjust='sidak')
posthoc.loc[posthoc['p-corr'] < 0.05]

### Left

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False].copy()

var = 'LeftSignal_dFF_zscore_filtered'

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan

data = data.dropna(subset=['LeftSignal_dFF_zscore_filtered'])

sns.lineplot(x = "TimeFromDrinkRound", y = var,  data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = "Sex",
             hue_order= ['Male','Female'],
             style = 'Fluid',
             style_order= ['Water','Alcohol'],
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Left.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv=var, between=['TimeFromDrinkRound','Sex','Fluid'], data = data,  detailed=True)

In [ ]:
posthoc = pg.pairwise_tests(dv=var, between=['TimeFromDrinkRound',"Sex"], data = data, padjust='sidak')
posthoc.loc[(posthoc['Contrast'] == "TimeFromDrinkRound * Sex") & (posthoc['p-corr'] < 0.05)].sort_values(by = 'TimeFromDrinkRound')

In [ ]:
posthoc = pg.pairwise_tests(dv=var, between=['TimeFromDrinkRound',"Fluid"], data = data, padjust='sidak')
posthoc.loc[(posthoc['Contrast'] == "TimeFromDrinkRound * Fluid") & (posthoc['p-corr'] < 0.05)].sort_values(by = 'TimeFromDrinkRound')

In [ ]:
var = 'LeftSignal_dFF_zscore_filtered'

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False].copy()

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Fluid', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= True, jitter=True,)
sns.pointplot(data = data, x = 'Fluid', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=.4, join = False)

plt.ylabel("Left AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Left_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv = var, between=['Sex','Fluid'], data = data)

In [ ]:
pg.pairwise_tests(dv = var, between=['Sex','Fluid'], data = data, padjust='sidak')

In [ ]:
var = 'LeftSignal_dFF_zscore_filtered'

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False].copy()

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Fluid', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= True, jitter=True,)
sns.pointplot(data = data, x = 'Fluid', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=.4, join = False)

plt.ylabel("Left AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Left_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv = var, between=['Sex','Fluid'], data = data)

In [ ]:
var = 'LeftSignal_dFF_zscore_filtered'

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False].copy()

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Fluid', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= True, jitter=True,)
sns.pointplot(data = data, x = 'Fluid', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=.4, join = False)

plt.ylabel("Left AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Left_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv = var, between=['Sex','Fluid'], data = data)

In [ ]:
# by sex
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False].copy()

var = 'RightSignal_dFF_zscore_filtered'

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan

data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,  data = data,
             estimator=np.mean, errorbar='se', size = 1,
             hue = "Sex",
             hue_order= ['Male','Female'],
             style = 'Fluid',
             style_order= ['Water','Alcohol'],
            )

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()

printedhandles = (handles[:3] + handles[4:])
ax.legend(handles = printedhandles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('Right.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv=var, between=['TimeFromDrinkRound','Sex','Fluid'], data = data, detailed=True)

In [ ]:
posthoc = pg.pairwise_tests(dv=var, between=['TimeFromDrinkRound',"Sex"], data = data, padjust='sidak')
posthoc.loc[(posthoc['Contrast'] == "TimeFromDrinkRound * Sex") & (posthoc['p-corr'] < 0.05)].sort_values(by = 'TimeFromDrinkRound')

In [ ]:
posthoc = pg.pairwise_tests(dv=var, between=['TimeFromDrinkRound',"Fluid"], data = data, padjust='sidak')
posthoc.loc[(posthoc['Contrast'] == "TimeFromDrinkRound * Fluid") & (posthoc['p-corr'] < 0.05)].sort_values(by = 'TimeFromDrinkRound')

In [ ]:
var = 'RightSignal_dFF_zscore_filtered'

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False].copy()

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "time_to_peak"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Fluid', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= True, jitter=True,)
sns.pointplot(data = data, x = 'Fluid', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=.4, join = False)

plt.ylabel("Right AIC Inputs\nTime to Peak Amplitude (s)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(0,6)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Right_TTP.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv = var, between=['Sex','Fluid'], data = data)

In [ ]:
pg.pairwise_tests(dv = var, between=['Sex','Fluid'], data = data, padjust='sidak')

In [ ]:
var = 'RightSignal_dFF_zscore_filtered'

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False].copy()

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan

data = data.dropna(subset=[var])

data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "AUC"

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Fluid', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= True, jitter=True,)
sns.pointplot(data = data, x = 'Fluid', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=.4, join = False)

plt.ylabel("Right AIC Inputs\nAUC (Peak Amplitude to Drink End)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-15,15)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Right_AUC.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv = var, between=['Sex','Fluid'], data = data)

In [ ]:
var = 'RightSignal_dFF_zscore_filtered'

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False].copy()

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])


data.loc[data['DIDWeek'] == 4, 'Week'] = 'Week 4'
data.loc[data['DIDWeek'] == 5, 'Week'] = 'Week 5'
data.loc[data['DIDWeek'] == 6, 'Week'] = 'Week 6'

for i in data['ID'].unique():
  for j in data.loc[data['ID'] == i, 'Session'].unique():
    for k in data.loc[(data['ID'] == i) & (data['Session'] == j), 'DrinkNumber'].unique():
      if data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0] > 0:
        drinklength = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "DrinkLength"].unique()[0]
      else:
        break
      #filter down the lengths to find the peaks so your not peak finding past the end of the drink...
      data_new = data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= 0) & (data['TimeFromDrinkRound'] <= drinklength)].copy()

      xdata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), 'TimeFromDrinkRound'])
      ydata = np.array(data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k), var])

      max_value =  ydata.max()
      time_to_peak = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrinkRound'].iloc[0]

      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "time_to_peak"] = data_new.loc[(data_new['ID'] == i) & (data_new['Session'] == j) & (data_new['DrinkNumber'] == k) &  (data_new[var] == ydata.max()), 'TimeFromDrink'].iloc[0]/1000

      xdata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), "TimeFromDrinkRound"])
      ydata = np.array(data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k) & (data['TimeFromDrinkRound'] >= time_to_peak) & (data['TimeFromDrinkRound'] <= drinklength), var])

      area = scipy.integrate.trapezoid(ydata, xdata)
      # Print the decay coefficient
      data.loc[(data['ID'] == i) & (data['Session'] == j) & (data['DrinkNumber'] == k), "AUC"] = area
      

data = data.groupby(['ID','Session','DrinkNumber','Sex','Fluid']).max()
data = data.reset_index()

pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

data = data.sort_values(by="Week", ascending = True)

sns.stripplot(data = data, x = 'Fluid', y = var, hue = "Sex", hue_order=['Male','Female'], alpha = 0.5, size = 1,  dodge= True, jitter=True,)
sns.pointplot(data = data, x = 'Fluid', y = var, hue = "Sex", errorbar = 'se', hue_order=['Male','Female'], scale = 0.5, errwidth = 0.5, dodge=.4, join = False)

plt.ylabel("Right AIC Inputs\n Peak Amplitude (Max zscore ΔF/F)", size = 5)
plt.xlabel(" ", size = 5)

plt.ylim(-3,10)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[:], labels[2:], frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, handletextpad= 0)

plt.tight_layout()
plt.savefig('Right_MAX.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv = var, between=['Sex','Fluid'], data = data)

In [ ]:
pg.pairwise_tests(dv = var, between=['Sex','Fluid'], data = data, padjust='sidak')

## Figure 6

In [ ]:
# by sex
pagewidth = 180/25.4
figwidth = pagewidth*0.75
figheight = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()
handles = []

data = FP_FINAL.loc[(FP_FINAL['ID'] == "P7") & (FP_FINAL['Session'] == 17)]
adjust = data['Timestamp'].min()

for i in data.loc[data['DrinkNumber'].notnull(), "DrinkNumber"].unique():
    linetime = data.loc[data['SYNC_Time'] < data.loc[(data['TimeFromDrink'] == 0) & (data['DrinkNumber'] == i), "SYNC_Time"].iloc[0], 'Timestamp'].max()
    plt.vlines(x = linetime - adjust , ymin = -1.85, ymax = 8.15, linestyles = 'dashed', color = 'black', alpha = 0.5, linewidth = 0.5)

data['TimestampPRETYY'] = data['Timestamp'] - data['Timestamp'].min()

sns.lineplot(x = "TimestampPRETYY", y = 'LeftSignal_dFF_zscore', data = data.loc[data['Timestamp'] > 1000])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:] + handles[:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('WholeSessionLeft.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

In [ ]:
# by sex
pagewidth = 180/25.4
figwidth = pagewidth*0.75
figheight = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()
handles = []

data = FP_FINAL.loc[(FP_FINAL['ID'] == "P7") & (FP_FINAL['Session'] == 17)]
adjust = data['Timestamp'].min()

for i in data.loc[data['DrinkNumber'].notnull(), "DrinkNumber"].unique():
    linetime = data.loc[data['SYNC_Time'] < data.loc[(data['TimeFromDrink'] == 0) & (data['DrinkNumber'] == i), "SYNC_Time"].iloc[0], 'Timestamp'].max()
    plt.vlines(x = linetime - adjust , ymin = -1.85, ymax = 8.15, linestyles = 'dashed', color = 'black', alpha = 0.5, linewidth = 0.5)

data['TimestampPRETYY'] = data['Timestamp'] - data['Timestamp'].min()

sns.lineplot(x = "TimestampPRETYY", y = 'RightSignal_dFF_zscore', data = data.loc[data['Timestamp'] > 1000], color = '#ff7f0e')

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:] + handles[:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('WholeSessionRIGHT.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()


In [ ]:
cocktailmenu = pd.DataFrame()

for i in FP_FINAL['ID'].unique():
    for j in FP_FINAL.loc[(FP_FINAL['ID'] == i), 'Session'].unique():
        drinklist = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j) & (FP_FINAL['DrinkNumber'].notnull()), 'DrinkNumber'].unique()
        menu = pd.DataFrame(data=drinklist)
        menu['ID'] = i
        menu['Session'] = j
        menu['Sex'] = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j), "Sex"].unique()[0]
        menu['Fluid'] = FP_FINAL.loc[(FP_FINAL['ID'] == i) & (FP_FINAL['Session'] == j), "Fluid"].unique()[0]
            
        cocktailmenu = pd.concat([cocktailmenu, menu])

cocktailmenu = cocktailmenu.rename(columns={0: 'DrinkNumber'})

pagewidth = 180/25.4
pagewidth = 180/25.4
figwidth = pagewidth*0.75
figheight = figwidth/2
figwidth = 2


f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

sns.histplot(data=cocktailmenu, x='DrinkNumber')
#ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.ylabel("Count", size = 5)
plt.xlabel("Drinks", size = 5)

plt.ylim([0,250])

plt.xticks([0, cocktailmenu['DrinkNumber'].quantile(.25), cocktailmenu['DrinkNumber'].quantile(.50), cocktailmenu['DrinkNumber'].quantile(.75), cocktailmenu['DrinkNumber'].max()])
plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('DrinkDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

In [ ]:
pagewidth = 180/25.4
pagewidth = 180/25.4
figwidth = pagewidth*0.75
figheight = figwidth/2
figwidth = 2

f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

sns.histplot(data=FP_FINAL.loc[FP_FINAL['DrinkLength'].notnull(), "DrinkLength"].unique())

plt.ylabel("Count", size = 5)
plt.xlabel("Drink Length (s)", size = 5)

plt.ylim([0,250])

plt.xticks([0, FP_FINAL['DrinkLength'].quantile(.25), FP_FINAL['DrinkLength'].quantile(.50), FP_FINAL['DrinkLength'].quantile(.75), FP_FINAL['DrinkLength'].max()], rotation=45, ha="right")
plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

plt.tight_layout()
plt.savefig('DrinkLengthDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

In [ ]:
# by sex
pagewidth = 180/25.4
figwidth = pagewidth*0.75
figheight = figwidth/2
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()
handles = []

FP_FINAL.loc[FP_FINAL['DrinkNumber'] < cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[FP_FINAL['DrinkNumber'].isnull() == False]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan

#combine data columns into one column
data['Sex-Fluid'] = data['Sex'] + ' - ' + data["Fluid"]
data = pd.melt(data, id_vars=['ID','Session','DrinkNumber','Sex','Fluid','Sex-Fluid','TimeFromDrinkRound','DrinkNumDist'], value_vars=['RightSignal_dFF_zscore_filtered','LeftSignal_dFF_zscore_filtered'])
data['Sex-Fluid-Side'] = data['Sex-Fluid'] + '-' + data['variable']
data = data.reset_index()

plt.hlines(y = 0 , xmin = -5, xmax = 5, linestyles = 'dashed', color = 'black', alpha = 0.5, linewidth = 0.5)
plt.vlines(x = 0 , ymin = -0.95, ymax = 1.45, linestyles = 'dashed', color = 'black', alpha = 0.5, linewidth = 0.5)

sns.lineplot(x = "TimeFromDrinkRound", y = "value", 
             hue = 'DrinkNumDist',
             style = 'variable',
             data = data.loc[data['Sex'] == "Male"],
             #hue_order = ['Male - Water','Female - Water','Male - Alcohol', 'Female - Alcohol'],
             style_order=['LeftSignal_dFF_zscore_filtered','RightSignal_dFF_zscore_filtered'],
             estimator=np.mean, errorbar='se', size = 1)

plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("zscore ΔF/F", size = 5)
plt.xlabel("Time from Bout Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
#plt.savefig('RightVLeft.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv='value', between=['DrinkNumDist','Fluid','TimeFromDrinkRound','variable'], data = data, detailed=True)

In [ ]:
data

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkNumber'] < cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False)]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])
data = data.loc[data['DrinkNumDist'].notnull()]

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkNumDist',
             hue_order=['Drinks 1 - 5','Drinks 6 - 9','Drinks 10 - 17','Drinks 18+'],
             errorbar='se',
             data = data)
             
plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title_fontsize = 5)

plt.tight_layout()
#plt.savefig('LeftMaleWaterDrink_NUM_Dist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

data['DrinkNumnDistDUMMY'] = pd.factorize(data['DrinkNumDist'])[0]

pg.rm_anova(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, detailed=True)

## Figure 6 - Figure Supplement 2

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkNumber'] < cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water") &
                    (FP_FINAL['Sex'] == "Male")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])
data = data.loc[data['DrinkNumDist'].notnull()]

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkNumDist',
             hue_order=['Drinks 1 - 5','Drinks 6 - 9','Drinks 10 - 17','Drinks 18+'],
             errorbar='se',
             data = data)
             
plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title_fontsize = 5)

plt.tight_layout()
plt.savefig('LeftMaleWaterDrink_NUM_Dist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

data['DrinkNumnDistDUMMY'] = pd.factorize(data['DrinkNumDist'])[0]

pg.rm_anova(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water") &
                    (FP_FINAL['Sex'] == "Male")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])
data = data.loc[data['DrinkNumDist'].notnull()]

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkNumDist',
             hue_order=['Drinks 1 - 5','Drinks 6 - 9','Drinks 10 - 17','Drinks 18+'],
             errorbar='se',
             data = data)
             
plt.ylim(-1.25,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title_fontsize = 5)

plt.tight_layout()
plt.savefig('RightMaleWaterDrink_NUM_Dist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

data['DrinkNumnDistDUMMY'] = pd.factorize(data['DrinkNumDist'])[0]

pg.rm_anova(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkNumber'] < cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol") &
                    (FP_FINAL['Sex'] == "Male")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])
data = data.loc[data['DrinkNumDist'].notnull()]

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkNumDist',
             hue_order=['Drinks 1 - 5','Drinks 6 - 9','Drinks 10 - 17','Drinks 18+'],
             errorbar='se',
             data = data)
             
plt.ylim(-1,2)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title_fontsize = 5)

plt.tight_layout()
plt.savefig('LeftMaleAlcoholDrink_NUM_Dist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

data['DrinkNumnDistDUMMY'] = pd.factorize(data['DrinkNumDist'])[0]

pg.rm_anova(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, detailed=True)

In [ ]:
posthoc = pg.pairwise_tests(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, padjust='sidak')
posthoc.loc[(posthoc['Contrast'] == "DrinkNumnDistDUMMY") & (posthoc['p-unc'] < 0.05)]

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkNumber'] < cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol") &
                    (FP_FINAL['Sex'] == "Male")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])
data = data.loc[data['DrinkNumDist'].notnull()]

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkNumDist',
             hue_order=['Drinks 1 - 5','Drinks 6 - 9','Drinks 10 - 17','Drinks 18+'],
             errorbar='se',
             data = data)
             
plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title_fontsize = 5)

plt.tight_layout()
plt.savefig('RightMaleAlcoholDrink_NUM_Dist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

data['DrinkNumnDistDUMMY'] = pd.factorize(data['DrinkNumDist'])[0]

pg.rm_anova(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkNumber'] < cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water") &
                    (FP_FINAL['Sex'] == "Female")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])
data = data.loc[data['DrinkNumDist'].notnull()]

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkNumDist',
             hue_order=['Drinks 1 - 5','Drinks 6 - 9','Drinks 10 - 17','Drinks 18+'],
             errorbar='se',
             data = data)
             
plt.ylim(-1.1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title_fontsize = 5)

plt.tight_layout()
plt.savefig('LeftFEMaleWaterDrink_NUM_Dist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

data['DrinkNumnDistDUMMY'] = pd.factorize(data['DrinkNumDist'])[0]

pg.rm_anova(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water") &
                    (FP_FINAL['Sex'] == "Female")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])
data = data.loc[data['DrinkNumDist'].notnull()]

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkNumDist',
             hue_order=['Drinks 1 - 5','Drinks 6 - 9','Drinks 10 - 17','Drinks 18+'],
             errorbar='se',
             data = data)
             
plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title_fontsize = 5)

plt.tight_layout()
plt.savefig('RightFEMaleWaterDrink_NUM_Dist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

data['DrinkNumnDistDUMMY'] = pd.factorize(data['DrinkNumDist'])[0]

pg.rm_anova(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkNumber'] < cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol") &
                    (FP_FINAL['Sex'] == "Female")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])
data = data.loc[data['DrinkNumDist'].notnull()]

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkNumDist',
             hue_order=['Drinks 1 - 5','Drinks 6 - 9','Drinks 10 - 17','Drinks 18+'],
             errorbar='se',
             data = data)
             
plt.ylim(-1.1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title_fontsize = 5)

plt.tight_layout()
plt.savefig('LeftFEMaleAlcoholDrink_NUM_Dist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

data['DrinkNumnDistDUMMY'] = pd.factorize(data['DrinkNumDist'])[0]

pg.rm_anova(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.25), "DrinkNumDist"] = f"Drinks 1 - {round(cocktailmenu['DrinkNumber'].quantile(.25))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.25)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.5)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.25))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.5)))}"
FP_FINAL.loc[(FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.5)) & (FP_FINAL['DrinkNumber'] <= cocktailmenu['DrinkNumber'].quantile(.75)), "DrinkNumDist"] = f"Drinks {(round(cocktailmenu['DrinkNumber'].quantile(.5))+1)} - {(round(cocktailmenu['DrinkNumber'].quantile(.75)))}"
FP_FINAL.loc[FP_FINAL['DrinkNumber'] > cocktailmenu['DrinkNumber'].quantile(.75), "DrinkNumDist"] = f"Drinks {round(cocktailmenu['DrinkNumber'].quantile(.75)+1)}+"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol") &
                    (FP_FINAL['Sex'] == "Female")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])
data = data.loc[data['DrinkNumDist'].notnull()]

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkNumDist',
             hue_order=['Drinks 1 - 5','Drinks 6 - 9','Drinks 10 - 17','Drinks 18+'],
             errorbar='se',
             data = data)
             
plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title_fontsize = 5)

plt.tight_layout()
plt.savefig('RightFEMaleAlcoholDrink_NUM_Dist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

data['DrinkNumnDistDUMMY'] = pd.factorize(data['DrinkNumDist'])[0]

pg.rm_anova(dv=var, within=['DrinkNumnDistDUMMY','TimeFromDrinkRound'], subject='ID', data = data, detailed=True)

## Figure 6 - Figure Supplement 1

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.25), "DrinkDist"] = f"0 - {FP_FINAL['DrinkLength'].quantile(.25).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.25)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.5)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.25).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.5).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.5)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.75)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.5).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.75).round(3)} (s)"
FP_FINAL.loc[FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.75), "DrinkDist"] = f"{((FP_FINAL['DrinkLength'].quantile(.75).round(3))+0.001).round(3)}+ (s)"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water") &
                    (FP_FINAL['Sex'] == "Male")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkDist',
             hue_order=['0 - 0.434 (s)','0.435 - 1.421 (s)','1.422 - 3.683 (s)','3.684+ (s)'],
             errorbar='se',
             data = data)
             
plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title = 'Drink Duration', title_fontsize = 5)

plt.tight_layout()
plt.savefig('LeftMaleWaterDrinkDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv=var, within=['DrinkDist','TimeFromDrinkRound'], subject='ID', data = data.loc[data['DrinkDist'].notnull()], detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.25), "DrinkDist"] = f"0 - {FP_FINAL['DrinkLength'].quantile(.25).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.25)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.5)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.25).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.5).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.5)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.75)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.5).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.75).round(3)} (s)"
FP_FINAL.loc[FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.75), "DrinkDist"] = f"{((FP_FINAL['DrinkLength'].quantile(.75).round(3))+0.001).round(3)}+ (s)"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water") &
                    (FP_FINAL['Sex'] == "Female")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkDist',
             hue_order=['0 - 0.434 (s)','0.435 - 1.421 (s)','1.422 - 3.683 (s)','3.684+ (s)'],
             errorbar='se',
             data = data)
             
plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title = 'Drink Duration', title_fontsize = 5)

plt.tight_layout()
plt.savefig('LeftFEMaleWaterDrinkDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv=var, within=['DrinkDist','TimeFromDrinkRound'], subject='ID', data = data.loc[data['DrinkDist'].notnull()], detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.25), "DrinkDist"] = f"0 - {FP_FINAL['DrinkLength'].quantile(.25).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.25)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.5)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.25).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.5).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.5)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.75)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.5).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.75).round(3)} (s)"
FP_FINAL.loc[FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.75), "DrinkDist"] = f"{((FP_FINAL['DrinkLength'].quantile(.75).round(3))+0.001).round(3)}+ (s)"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol") &
                    (FP_FINAL['Sex'] == "Male")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkDist',
             hue_order=['0 - 0.434 (s)','0.435 - 1.421 (s)','1.422 - 3.683 (s)','3.684+ (s)'],
             errorbar='se',
             data = data)
             
plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title = 'Drink Duration', title_fontsize = 5)

plt.tight_layout()
plt.savefig('LeftMaleAlcoholDrinkDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv=var, within=['DrinkDist','TimeFromDrinkRound'], subject='ID', data = data.loc[data['DrinkDist'].notnull()], detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "LeftSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.25), "DrinkDist"] = f"0 - {FP_FINAL['DrinkLength'].quantile(.25).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.25)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.5)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.25).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.5).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.5)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.75)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.5).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.75).round(3)} (s)"
FP_FINAL.loc[FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.75), "DrinkDist"] = f"{((FP_FINAL['DrinkLength'].quantile(.75).round(3))+0.001).round(3)}+ (s)"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol") &
                    (FP_FINAL['Sex'] == "Female")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkDist',
             hue_order=['0 - 0.434 (s)','0.435 - 1.421 (s)','1.422 - 3.683 (s)','3.684+ (s)'],
             errorbar='se',
             data = data)
             
plt.ylim(-1,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Left AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title = 'Drink Duration', title_fontsize = 5)

plt.tight_layout()
plt.savefig('LeftFEMaleAlcoholDrinkDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv=var, between=['DrinkDist','TimeFromDrinkRound'], data = data)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.25), "DrinkDist"] = f"0 - {FP_FINAL['DrinkLength'].quantile(.25).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.25)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.5)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.25).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.5).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.5)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.75)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.5).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.75).round(3)} (s)"
FP_FINAL.loc[FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.75), "DrinkDist"] = f"{((FP_FINAL['DrinkLength'].quantile(.75).round(3))+0.001).round(3)}+ (s)"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol") &
                    (FP_FINAL['Sex'] == "Male")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkDist',
             hue_order=['0 - 0.434 (s)','0.435 - 1.421 (s)','1.422 - 3.683 (s)','3.684+ (s)'],
             errorbar='se',
             data = data)
             
plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title = 'Drink Duration', title_fontsize = 5)

plt.tight_layout()
#lt.savefig('RightMaleAlcoholDrinkDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv=var, between=['DrinkDist','TimeFromDrinkRound'], data = data)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.25), "DrinkDist"] = f"0 - {FP_FINAL['DrinkLength'].quantile(.25).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.25)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.5)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.25).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.5).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.5)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.75)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.5).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.75).round(3)} (s)"
FP_FINAL.loc[FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.75), "DrinkDist"] = f"{((FP_FINAL['DrinkLength'].quantile(.75).round(3))+0.001).round(3)}+ (s)"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Alcohol") &
                    (FP_FINAL['Sex'] == "Female")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkDist',
             hue_order=['0 - 0.434 (s)','0.435 - 1.421 (s)','1.422 - 3.683 (s)','3.684+ (s)'],
             errorbar='se',
             data = data)
             
plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5, title = 'Drink Duration', title_fontsize = 5)

plt.tight_layout()
plt.savefig('RightFEMaleAlcoholDrinkDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.anova(dv=var, between=['DrinkDist','TimeFromDrinkRound'], data = data)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.25), "DrinkDist"] = f"0 - {FP_FINAL['DrinkLength'].quantile(.25).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.25)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.5)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.25).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.5).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.5)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.75)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.5).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.75).round(3)} (s)"
FP_FINAL.loc[FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.75), "DrinkDist"] = f"{((FP_FINAL['DrinkLength'].quantile(.75).round(3))+0.001).round(3)}+ (s)"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water") &
                    (FP_FINAL['Sex'] == "Male")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkDist',
             hue_order=['0 - 0.434 (s)','0.435 - 1.421 (s)','1.422 - 3.683 (s)','3.684+ (s)'],
             errorbar='se',
             data = data)
             
plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('RightMaleWaterDrinkDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv=var, within=['DrinkDist','TimeFromDrinkRound'], subject='ID', data = data.loc[data['DrinkDist'].notnull()], detailed=True)

In [ ]:
pagewidth = 180/25.4
figwidth = pagewidth/3
figheight = figwidth
figwidth = figwidth + (figwidth/2)
f, ax = plt.subplots(figsize=(figwidth,figheight))
sns.despine()

var = "RightSignal_dFF_zscore_filtered"

FP_FINAL.loc[FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.25), "DrinkDist"] = f"0 - {FP_FINAL['DrinkLength'].quantile(.25).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.25)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.5)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.25).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.5).round(3)} (s)"
FP_FINAL.loc[(FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.5)) & (FP_FINAL['DrinkLength'] <= FP_FINAL['DrinkLength'].quantile(.75)), "DrinkDist"] = f"{(FP_FINAL['DrinkLength'].quantile(.5).round(3)+0.001)} - {FP_FINAL['DrinkLength'].quantile(.75).round(3)} (s)"
FP_FINAL.loc[FP_FINAL['DrinkLength'] > FP_FINAL['DrinkLength'].quantile(.75), "DrinkDist"] = f"{((FP_FINAL['DrinkLength'].quantile(.75).round(3))+0.001).round(3)}+ (s)"

data = FP_FINAL.loc[(FP_FINAL['DrinkNumber'].isnull() == False) &
                    (FP_FINAL['Fluid'] == "Water") &
                    (FP_FINAL['Sex'] == "Female")]

data.loc[(data['ID'] == "P9") & (data['Session'] == 7), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 8), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 10), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P8") & (data['Session'] == 16), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P6") & (data['Session'] == 18), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 20), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P9") & (data['Session'] == 24), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P7") & (data['Session'] == 30), 'LeftSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P1") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data.loc[(data['ID'] == "P13") & (data['Session'] == 25), 'RightSignal_dFF_zscore_filtered'] = np.nan
data = data.dropna(subset=[var])

sns.lineplot(x = "TimeFromDrinkRound", y = var,
             hue = 'DrinkDist',
             hue_order=['0 - 0.434 (s)','0.435 - 1.421 (s)','1.422 - 3.683 (s)','3.684+ (s)'],
             errorbar='se',
             data = data)
             
plt.ylim(-1.2,1.5)
plt.xticks([-5,-4,-3,-2,-1,0,1,2,3,4,5])

plt.ylabel("Right AIC Inputs\nzscore ΔF/F", size = 5)
plt.xlabel("Time from Drink Start (s)", size = 5)

plt.tick_params(axis='both', which='major', labelsize=5)

for _,s in ax.spines.items():
    s.set_linewidth(0.5)
    
ax.tick_params(width=0.5, length = 2)

handles, labels = ax.get_legend_handles_labels()
handles = (handles[:5] + handles[6:])
ax.legend(handles = handles, frameon=False, loc = 'upper left', prop = {"size":5}, markerscale = 0.5)

plt.tight_layout()
plt.savefig('RightFEMaleWaterDrinkDist.pdf', transparent = True, dpi = 1200)
plt.show()
plt.clf()

pg.rm_anova(dv=var, within=['DrinkDist','TimeFromDrinkRound'], subject='ID', data = data.loc[data['DrinkDist'].notnull()], detailed=True)